In [6]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

import helpers as hp

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download rent data

In [ ]:
base_url = 'http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/'
file_names = [
    'detailed-lodged-bonds.csv',
    'detailed-mean-rents.csv',
    'detailed-geo-mean-rents.csv',
]

for file_name in file_names:
    url = base_url + file_name
    print("Getting and saving", url)
    r = requests.get(url)
    path = hp.DATA_DIR/'collected'/file_name
    with path.open('w') as tgt:
        tgt.write(r.text)


# Prepare rent data

In [22]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    hp.DATA_DIR/'collected'/'detailed-lodged-bonds.csv',
    hp.DATA_DIR/'collected'/'detailed-mean-rents.csv',
    hp.DATA_DIR/'collected'/'detailed-geo-mean-rents.csv',
]
names = [
    'rent_count', 
    'rent_mean', 
    'rent_geo_mean',
]
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
238,500202,Flat or Apartment,1,1995-03-01,8.0,82.0,82.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
258,500202,Flat or Apartment,1,1996-03-01,5.0,79.0,79.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
261,500202,House,3,1996-03-01,5.0,123.0,118.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
286,500202,House,3,1997-06-01,5.0,154.0,152.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
311,500202,House,3,1998-09-01,7.0,139.0,137.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo


In [23]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2017-06-01', '2017-09-01']

# Explore rents

In [24]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [4]:
# Slice in time and aggregate 

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_count'].dropna().shape[0]/group['rent_count'].shape[0]
    return pd.Series(d)

agg_rents = hp.aggregate_rents(rents, '2017-03-01')
agg_rents.head()

,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,175.0,211.264853,213.931429,Christchurch City
1,Addington,2,Canterbury,104.0,322.500943,328.201923,Christchurch City
2,Addington,3,Canterbury,96.0,409.234255,416.583333,Christchurch City
3,Addington,4,Canterbury,5.0,488.000000,494.000000,Christchurch City
4,Addington,5+,Canterbury,NaN,NaN,NaN,Christchurch City


In [25]:
# What fraction of rental data do we have by #bedrooms?

date = '2017-03-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


census area units
  #bedrooms  hit_frac
0         1  0.378125
1         2  0.656977
2         3  0.840456
3         4  0.599424
4        5+  0.128205
rental area units
  #bedrooms  hit_frac
0         1  0.757576
1         2  0.949495
2         3  1.000000
3         4  0.919192
4        5+  0.272727


# Select latest two quarters and slice into regional chunks

In [28]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents[rents['quarter'] >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents[rents['region'] == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2017-06-01' '2017-09-01'] #rows= 5452
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2017-06-01' '2017-09-01'] #rows= 2522
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2017-06-01' '2017-09-01'] #rows= 2412
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [37]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.jsonize_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

** auckland 
 {'Henderson': {'3': 495.0, '4': 604.0, '2': 405.0, '1': 296.0}, 'Kelston': {'3': 495.0, '4': 555.0, '2': 378.0, '1': 316.0}, 'Massey/Royal Heights': {'3': 483.0, '4': 571.0, '2': 395.0, '1': None}, 'Panmure': {'3': 522.0, '4': None, '2': 454.0, '1': 289.0}, 'Glen Innes/Pt England/Wai O Taiki': {'3': 570.0, '4': None, '2': None, '1': None}, 'Wellsford/Warkworth/Helensville': {'3': 463.0, '4': 549.0, '2': 403.0, '1': 284.0}, 'Wattle Downs/Conifer Grove': {'3': 520.0, '4': 636.0, '2': 416.0, '1': None}, 'Central West': {'3': 681.0, '4': 719.0, '2': 513.0, '1': 378.0}, 'Half Moon Bay/Farm Cove': {'3': 528.0, '4': 728.0, '2': 474.0, '1': None}, 'West Harbour': {'3': 559.0, '4': 710.0, '2': 475.0, '1': None}, 'Howick': {'3': 565.0, '4': 752.0, '2': 466.0, '1': 307.0}, 'Mangere Bridge/Airport': {'3': 513.0, '4': 610.0, '2': 441.0, '1': 353.0}, 'Botony Downs': {'3': 582.0, '4': 672.0, '2': None, '1': None}, 'Penrose/Mt Wellington South': {'3': 542.0, '4': 603.0, '2': 435.0, '1': 

** wellington 
 {'Western Hills/Haywards': {'3': 444.0, '4': 548.0, '2': 347.0, '1': 197.0}, 'Kilbirnie/Lyall Bay': {'3': 586.0, '4': None, '2': 435.0, '1': 340.0}, 'Mt Cook': {'3': 584.0, '4': 673.0, '2': 496.0, '1': 251.0}, 'Totara Park/Maoribank/Te Marua': {'3': 430.0, '4': None, '2': 291.0, '1': None}, 'Trentham West/Eldersley/Clouston Park': {'3': 449.0, '4': None, '2': 285.0, '1': 165.0}, 'Trentham North/Wallaceville': {'3': 436.0, '4': None, '2': 287.0, '1': 152.0}, 'Petone/Esplanade': {'3': 535.0, '4': None, '2': 419.0, '1': 242.0}, 'Karori': {'3': 588.0, '4': None, '2': None, '1': 295.0}, 'Ngaio/Kaiwharawhara/Wilton': {'3': 552.0, '4': 676.0, '2': 391.0, '1': 306.0}, 'Wadestown/Thordon': {'3': 592.0, '4': None, '2': 507.0, '1': 340.0}, 'Brooklyn': {'3': 617.0, '4': None, '2': 447.0, '1': 352.0}, 'Oriental Bay/Seatoun': {'3': 757.0, '4': None, '2': 633.0, '1': 361.0}, 'Hutt Central/Waterloo': {'3': 511.0, '4': None, '2': 399.0, '1': 263.0}, 'Kelburn/Aro Valley': {'3': 550.0, '4